# Fingernails Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the necessary modules.
from google.colab import drive
import os

# Check if the directory is empty. If it isn't list the contents so the user can diagnose.
if os.path.exists('/content/drive') and os.listdir('/content/drive'):
  print("The /content/drive directory is not empty and contains the following files:")
  for item in os.listdir('/content/drive'):
    print(item)
else:
  # If the directory is empty or doesn't exist, proceed with mounting.
  drive.mount('/content/drive')

The /content/drive directory is not empty and contains the following files:
MyDrive
.file-revisions-by-id
.shortcut-targets-by-id
.Trash-0
.Encrypted


In [ ]:
!cp "/content/drive/MyDrive/Fingernails (1).zip" "/content/"

cp: cannot stat '/content/drive/MyDrive/Fingernails (1).zip': No such file or directory


In [ ]:
!unzip "/content/Fingernails.zip"

Archive:  /content/Fingernails.zip
   creating: Fingernails/
   creating: Fingernails/Fingernails/
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (10).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (11).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (12).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (2).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (3).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (4).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (5).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (6).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (7).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (8).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007 (9).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-007.png  
  inflating: Fingernails/Fingernails/Anemic-Fin-008 (10).png  
  inflating: Fingernails/Fingernails/Anemic-Fin-008 (11).png  
  inflating: Fingernails/Fingern

In [ ]:
import os
import cv2

# Path to the unzipped folder
dataset_folder = '/content/Fingernails/Fingernails'

# Initialize lists to store images and labels
anemic_images = []
non_anemic_images = []

# Loop through all files in the folder
for filename in os.listdir(dataset_folder):
    if filename.startswith('Anemic'):
        img = cv2.imread(os.path.join(dataset_folder, filename))
        if img is not None:
            anemic_images.append(img)
    else:
        img = cv2.imread(os.path.join(dataset_folder, filename))
        if img is not None:
            non_anemic_images.append(img)

print(f"Loaded {len(anemic_images)} anemic images and {len(non_anemic_images)} non-anemic images.")

Loaded 2521 anemic images and 1739 non-anemic images.


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
import os
import zipfile
import shutil
import random
from pathlib import Path

#  CONFIGURATION
zip_path = "Fingernails.zip"  # Update this if needed
output_path = Path("Structured_Fingernails")  # Final structured output
extract_path = Path("Fingernails_Unzipped")
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# STEP 1: Extract the ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# STEP 2: Locate the actual image folder (assuming nested structure)
image_folder = extract_path / "Fingernails" / "Fingernails"

# STEP 3: Create folders
for split in ["train", "val", "test"]:
    for category in ["Anemic", "Non_Anemic"]:
        (output_path / split / category).mkdir(parents=True, exist_ok=True)

# STEP 4: Collect and label images
anemic_images = [f for f in image_folder.iterdir() if "Anemic" in f.name and f.is_file()]
non_anemic_images = [f for f in image_folder.iterdir() if "Anemic" not in f.name and f.is_file()]

# STEP 5: Split function
def split_data(images, train_ratio, val_ratio):
    random.shuffle(images)
    total = len(images)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)
    return images[:train_end], images[train_end:val_end], images[val_end:]

# STEP 6: Copy files to folders
def distribute(images, category):
    train_imgs, val_imgs, test_imgs = split_data(images, train_ratio, val_ratio)
    for img in train_imgs:
        shutil.copy(img, output_path / "train" / category / img.name)
    for img in val_imgs:
        shutil.copy(img, output_path / "val" / category / img.name)
    for img in test_imgs:
        shutil.copy(img, output_path / "test" / category / img.name)

distribute(anemic_images, "Anemic")
distribute(non_anemic_images, "Non_Anemic")

print(" Dataset structured and split:")
print(" - train/ (70%)")
print(" - val/   (20%)")
print(" - test/  (10%)")
print(f" Root folder: {output_path.resolve()}")


 Dataset structured and split:
 - train/ (70%)
 - val/   (20%)
 - test/  (10%)
 Root folder: /content/Structured_Fingernails


In [ ]:
dataset_path = "Structured_Fingernails"  # Update this path if needed

# Add augmentation for variation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes
print("Classes:", class_names)


Classes: ['Anemic', 'Non_Anemic']


In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)  # Binary classification
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Checkpoint setup
best_val_loss = float('inf')
checkpoint_path = "best_resnet18_model.pth"


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 189MB/s]


In [ ]:
def train_model(model, criterion, optimizer, num_epochs=200):
    global best_val_loss

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        preds_all, labels_all = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.float().unsqueeze(1).to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                preds_all.extend(torch.sigmoid(outputs).cpu().numpy())
                labels_all.extend(labels.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        preds_binary = [1 if p > 0.5 else 0 for p in preds_all]

        precision = precision_score(labels_all, preds_binary)
        recall = recall_score(labels_all, preds_binary)
        f1 = f1_score(labels_all, preds_binary)
        auc = roc_auc_score(labels_all, preds_all)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | Precision: {precision:.4f} | "
              f"Recall: {recall:.4f} | F1: {f1:.4f} | AUC: {auc:.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), checkpoint_path)


In [ ]:
train_model(model, criterion, optimizer, num_epochs=200)
print("Training completed. Best model saved to:", checkpoint_path)


Epoch 1/200 | Train Loss: 0.6503 | Val Loss: 0.6874 | Precision: 0.4909 | Recall: 0.7752 | F1: 0.6011 | AUC: 0.6404
Epoch 2/200 | Train Loss: 0.6400 | Val Loss: 0.6657 | Precision: 0.5190 | Recall: 0.3141 | F1: 0.3914 | AUC: 0.6131
Epoch 3/200 | Train Loss: 0.6557 | Val Loss: 0.6980 | Precision: 0.5500 | Recall: 0.0951 | F1: 0.1622 | AUC: 0.6000
Epoch 4/200 | Train Loss: 0.6428 | Val Loss: 0.8016 | Precision: 0.5357 | Recall: 0.3890 | F1: 0.4508 | AUC: 0.6177
Epoch 5/200 | Train Loss: 0.6331 | Val Loss: 0.6204 | Precision: 0.5954 | Recall: 0.5937 | F1: 0.5945 | AUC: 0.7112
Epoch 6/200 | Train Loss: 0.6146 | Val Loss: 0.6346 | Precision: 0.6211 | Recall: 0.4063 | F1: 0.4913 | AUC: 0.7046
Epoch 7/200 | Train Loss: 0.6156 | Val Loss: 0.6306 | Precision: 0.6468 | Recall: 0.4697 | F1: 0.5442 | AUC: 0.7239
Epoch 8/200 | Train Loss: 0.6181 | Val Loss: 0.6661 | Precision: 0.5888 | Recall: 0.3631 | F1: 0.4492 | AUC: 0.6849
Epoch 9/200 | Train Loss: 0.6084 | Val Loss: 0.6473 | Precision: 0.6438 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load best model
model.load_state_dict(torch.load("best_resnet18_model.pth"))
model.eval()

# Load test set
test_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'test'), transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluate
all_preds = []
all_probs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.float().unsqueeze(1).to(device)
        outputs = model(inputs)
        probs = torch.sigmoid(outputs).cpu().numpy()
        preds = (probs > 0.5).astype(int)

        all_probs.extend(probs.flatten())
        all_preds.extend(preds.flatten())
        all_labels.extend(labels.cpu().numpy().flatten())

# Metrics
acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds)
rec = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
auc = roc_auc_score(all_labels, all_probs)

# Display
print(f" Test Results (Like Table 3)")
print(f"Accuracy     : {acc:.4f}")
print(f"Precision    : {prec:.4f}")
print(f"Recall       : {rec:.4f}")
print(f"F1 Score     : {f1:.4f}")
print(f"AUC ROC      : {auc:.4f}")


 Test Results (Like Table 3)
Accuracy     : 0.8435
Precision    : 0.8000
Recall       : 0.8229
F1 Score     : 0.8113
AUC ROC      : 0.9256
